In [ ]:
#@title setup
import os
clone_drive = False # @param {type:"boolean"}
if clone_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    # will still work without cloning.

if not os.path.exists ("/content/drive"):
    base_path = "/content/drba"
else:
    base_path = "/content/drive/MyDrive/drba"

inputdir = os.path.join(base_path, "input")
outputdir = os.path.join(base_path, "output")

for path in [base_path, inputdir, outputdir]:
    if not os.path.exists(path):
        os.makedirs(path)

from IPython.display import clear_output
clear_output()

!git clone https://github.com/routineLife1/DRBA.git

print("upload file at", inputdir)

In [ ]:
#@title inference
#@markdown TIMES (default 2)
TIMES = 2 # @param {type:"slider", min:2, max:16, step:1}
Model = "rife" # @param ["rife", "gmfss", "gmfss_union"]
#@markdown flow scale, generally use 1.0 with 1080p and 0.5 with 4K resolution for best result
SCALE = "1.0" # @param ["0.25", "0.5", "1.0", "2.0", "4.0"]
#@markdown clear output messeges
clear_omsg = True # @param {type:"boolean"}

import os
import cv2
import shutil
framepers = round((24000/1001) * TIMES, 3)
rnm = Model.upper()

if not os.path.exists ("/content/drive"):
    base_path = "/content/drba"
else:
    base_path = "/content/drive/MyDrive/drba"
inputdir = os.path.join(base_path, "input")
outputdir = os.path.join(base_path, "output")

file_list = [filename for filename in os.listdir(inputdir) if filename.endswith(('.mp4', '.mov', '.mkv', '.avi', '.flv'))]
if not file_list:
    print(f"No files.\nPlease upload file at", inputdir)
else:
    total_files = len(file_list)
    for i, filename in enumerate(file_list):
        input_file = os.path.join(inputdir, filename)

        cap = cv2.VideoCapture(input_file)
        fps = cap.get(cv2.CAP_PROP_FPS)
        fps = round(fps, 3)
        in_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        in_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        cap.release()

        current_file_num = i + 1
        print(f"\tProcessing file {current_file_num} of {total_files}.")

        os.system(f'ffmpeg -loglevel error -y -i "{input_file}" -c:v copy -c:a copy /content/temp.mov')

        # inference
        print(f"Running inference...")
        %cd /content/DRBA
        !python infer.py -i /content/temp.mov -o /content/interpolated-temp.mov -t {TIMES} -m {Model} -scale {SCALE} -s -st 0.3
        # alternatively, you can cap fps to 60
        # !python infer.py -i /content/temp.mov -o /content/interpolated-temp.mov -fps 60 -m {Model} -scale {SCALE} -s -st 0.3

        # clean up
        !rm -rf /content/temp.mov

        print(f"\t done {filename}.\n")

        dst_dir = outputdir
        old_file = [name for name in os.listdir(dst_dir) if name.endswith('.mov') and name.startswith(f'{filename}_AFI-{rnm}-{framepers}fps_')]
        if not old_file:
            num_files = 0
        else:
            num_files = max(int(file.split('_')[-1].split('.')[0]) for file in old_file)
        new_file_name = f'{filename}-{rnm}-{framepers}fps_{str(num_files + 1).zfill(3)}.mov'
        src_file = ("/content/interpolated-temp.mov")
        dst_file = os.path.join(dst_dir, new_file_name)
        shutil.move(src_file, dst_file)

        if clear_omsg:
            from IPython.display import clear_output
            clear_output()


    print (f"task done.")